# Webscraping Glassdoor Reviews

In [259]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import time
import re

In [5]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}

In [263]:
# change user agent HERE
# page = requests.get('https://www.glassdoor.com/Reviews/Booking-com-Reviews-E256653.htm', headers=headers)
page = requests.get('https://www.glassdoor.com/Reviews/Booking-com-Reviews-E256653_P3.htm', headers=headers) # page 2
# page = requests.get('https://www.indeed.com/cmp/Booking.com/reviews')
soup = BeautifulSoup(page.content, 'lxml')

In [294]:
print(soup.prettify())
# looks like a hot mess

<!DOCTYPE html>
<html class="flex" lang="en" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraph.org/schema/">
 <head prefix="og: http://ogp.me/ns# fb: http://ogp.me/ns/fb# glassdoor: http://ogp.me/ns/fb/glassdoor#">
  <meta content="2,734 Booking.com reviews. A free inside look at company reviews and salaries posted anonymously by employees." name="description"/>
  <meta content="2,734 Booking.com reviews. A free inside look at company reviews and salaries posted anonymously by employees." name="og:description"/>
  <link href="https://www.glassdoor.com/Reviews/Booking-com-Reviews-E256653_P50.htm" rel="canonical"/>
  <link href="https://www.glassdoor.com/Reviews/Booking-com-Reviews-E256653_P49.htm" rel="prev"/>
  <link href="https://www.glassdoor.com/Reviews/Booking-com-Reviews-E256653_P51.htm" rel="next"/>
  <!-- because the getter clears the value -->
  <script>
   window.gdGlobals = window.gdGlobals ||
		[{
			'analyticsId':                      "UA-2595786-1",



Testing page 50,51 to look at company responses, as they get included in pros/cons list.

In [291]:
# page = requests.get('https://www.glassdoor.com/Reviews/Booking-com-Reviews-E256653_P50.htm', headers=headers) # page 2
# soup = BeautifulSoup(page.content, 'lxml')
print(len(get_titles(soup)))
print(len(get_ratings(soup)))
print(len(get_pros_cons(soup)[0]))
print(len(get_pros_cons(soup)[1]))

10
10
14
13


## Functions for retrieving titles, ratings, and pros/cons from Glassdoor reviews

In [262]:
# Obtain all review titles

def get_titles(soup):
    parsed_titles = soup.select('div h2 a')
    return [title.text for title in parsed_titles]


# Obtain all star ratings
# need to omit first rating since it refers to overall rating of the company

def get_ratings(soup):
    parsed_ratings = soup.find_all('span', class_="value-title")
    return [float(rating['title']) for rating in parsed_ratings][1:]
    

# Pros and Cons per page. Should be 20 total for 10 reviews. Pros and Cons are required fields when entering a review
# so there should not be any empty fields.

# returns a single tuple with a list of pros, and a list of cons
def get_pros_cons(soup):
    parsed_reviews = soup.find_all('p', class_="mt-0 mb-xsm v2__EIReviewDetailsV2__bodyColor v2__EIReviewDetailsV2__lineHeightLarge")
    reviews = [review.text for review in parsed_reviews]
    pros = reviews[0::2]
    cons = reviews[1::2]
    return pros,cons

# url = page 1 url of reviews
# start_pg = first page to scrape
# end_pg = last page to scrape
def scrape_reviews(url, company, start_pg, end_pg, df=pd.DataFrame({'title': [], 'rating':[], 'pros':[], 'cons':[]})):
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/718.93 (KHTML, like Gecko) Chrome/79.0.4927.49 Safari/489.72'}
    t0 = time.time()
    page_num = ''
    counter = 0
    error_pages = []

    for j in range(start_pg, end_pg+1):
        # only add page reference to url for 2nd page onwards
        if j !=1:
            page_num = f"_P{j}"
        time.sleep(1)
        
        # add '_P2' etc to end of url
        new_url = f"{url[:-4]}{page_num}.htm"
        page = requests.get(new_url, headers=headers)
        print(new_url)
        
        # using predefined functions to create lists of titles, ratings, pros, cons
        soup = BeautifulSoup(page.content, 'lxml')
        titles = get_titles(soup)
        ratings = get_ratings(soup)
        pros = get_pros_cons(soup)[0]
        cons = get_pros_cons(soup)[1]
        
        print('Array lengths: ', len(titles), len(ratings), len(pros), len(cons))
        
        # handle company responses, as they will get included in pros/cons
        try: 
            temp_df = pd.DataFrame({'title':titles, 'rating':ratings, 'pros':pros, 'cons':cons})
            df = df.append(temp_df, ignore_index=True)
        except ValueError:
            counter +=1
            print(f'Company responses on {counter} pages')
            error_pages.append(j)
            pass
        else:
            pass

        print('Dataframe shape: ', df.shape)
        
        #save the df every 50 pages just in case the script fails
        if j%50 == 0:
            df.to_csv('../data/reviews_TEMP.csv')
        
    # print number of pages with company responses
    print('\n', f'Errors on {len(error_pages)} pages:{error_pages}')
    
    # add a 'company' column to the dataframe
    df['company'] = company
    
    t1 = time.time()
    print(f'This operation took {(t1-t0)/60} minutes to scrape {end_pg-start_pg+1} pages at a pace of {(t1-t0)/(end_pg-start_pg+1)} sec/page.')
    
    return df

## Webscraping script to collect review information

In [252]:
# df = pd.DataFrame({'title': [], 'rating':[], 'pros':[], 'cons':[]})

,title,rating,pros,cons


In [265]:
df = scrape_reviews('https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97.htm','Best Buy',1,201)
df.pros = df.pros.map(lambda x: re.sub('\s+', ' ', x))
df.cons = df.cons.map(lambda x: re.sub('\s+', ' ', x))
df.to_csv('../data/reviews_best_buy.csv')

df = scrape_reviews('https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874.htm','Liberty Mutual',1,201)
df.pros = df.pros.map(lambda x: re.sub('\s+', ' ', x))
df.cons = df.cons.map(lambda x: re.sub('\s+', ' ', x))
df.to_csv('../data/reviews_liberty_mutual.csv')

df = scrape_reviews('https://www.glassdoor.com/Reviews/Merck-Reviews-E438.htm','Merck',1,201)
df.pros = df.pros.map(lambda x: re.sub('\s+', ' ', x))
df.cons = df.cons.map(lambda x: re.sub('\s+', ' ', x))
df.to_csv('../data/reviews_merck.csv')

# df = scrape_reviews('https://www.glassdoor.com/Reviews/Delta-Air-Lines-Reviews-E197.htm','Delta Airlines',1,201)
# df.pros = df.pros.map(lambda x: re.sub('\s+', ' ', x))
# df.cons = df.cons.map(lambda x: re.sub('\s+', ' ', x))
# df.to_csv('../data/reviews_delta.csv')



https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97.htm
Array lengths:  10 10 10 10
Dataframe shape:  (10, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P2.htm
Array lengths:  10 10 10 10
Dataframe shape:  (20, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P3.htm
Array lengths:  10 10 10 10
Dataframe shape:  (30, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P4.htm
Array lengths:  10 10 10 10
Dataframe shape:  (40, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P5.htm
Array lengths:  10 10 10 10
Dataframe shape:  (50, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P6.htm
Array lengths:  10 10 10 10
Dataframe shape:  (60, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P7.htm
Array lengths:  10 10 10 10
Dataframe shape:  (70, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P8.htm
Array lengths:  10 10 10 10
Dataframe shape:  (80, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P9.htm
Array lengths

https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P71.htm
Array lengths:  10 10 10 10
Dataframe shape:  (710, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P72.htm
Array lengths:  10 10 10 10
Dataframe shape:  (720, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P73.htm
Array lengths:  10 10 10 10
Dataframe shape:  (730, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P74.htm
Array lengths:  10 10 10 10
Dataframe shape:  (740, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P75.htm
Array lengths:  10 10 10 10
Dataframe shape:  (750, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P76.htm
Array lengths:  10 10 10 10
Dataframe shape:  (760, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P77.htm
Array lengths:  10 10 10 10
Dataframe shape:  (770, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P78.htm
Array lengths:  10 10 10 10
Dataframe shape:  (780, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P

https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P140.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1400, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P141.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1410, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P142.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1420, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P143.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1430, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P144.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1440, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P145.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1450, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P146.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1460, 4)
https://www.glassdoor.com/Reviews/Best-Buy-Reviews-E97_P147.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1470, 4)
https://www.glassdoor.com/Reviews/Best-B

https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P5.htm
Array lengths:  10 10 13 12
Company responses on 5 pages
Dataframe shape:  (0, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P6.htm
Array lengths:  10 10 11 11
Company responses on 6 pages
Dataframe shape:  (0, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P7.htm
Array lengths:  10 10 12 11
Company responses on 7 pages
Dataframe shape:  (0, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P8.htm
Array lengths:  10 10 12 11
Company responses on 8 pages
Dataframe shape:  (0, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P9.htm
Array lengths:  10 10 11 10
Company responses on 9 pages
Dataframe shape:  (0, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P10.htm
Array lengths:  10 10 10 10
Dataframe shape:  (10, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insura

https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P56.htm
Array lengths:  10 10 13 12
Company responses on 51 pages
Dataframe shape:  (50, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P57.htm
Array lengths:  10 10 12 12
Company responses on 52 pages
Dataframe shape:  (50, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P58.htm
Array lengths:  10 10 13 13
Company responses on 53 pages
Dataframe shape:  (50, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P59.htm
Array lengths:  10 10 12 12
Company responses on 54 pages
Dataframe shape:  (50, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P60.htm
Array lengths:  10 10 12 11
Company responses on 55 pages
Dataframe shape:  (50, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P61.htm
Array lengths:  10 10 11 11
Company responses on 56 pages
Dataframe shape:  (50, 4)
https://ww

Array lengths:  10 10 14 14
Company responses on 88 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P109.htm
Array lengths:  10 10 14 13
Company responses on 89 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P110.htm
Array lengths:  10 10 12 12
Company responses on 90 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P111.htm
Array lengths:  10 10 12 11
Company responses on 91 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P112.htm
Array lengths:  10 10 11 11
Company responses on 92 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P113.htm
Array lengths:  10 10 12 11
Company responses on 93 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P114.ht

Array lengths:  10 10 12 11
Company responses on 137 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P158.htm
Array lengths:  10 10 14 13
Company responses on 138 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P159.htm
Array lengths:  10 10 11 11
Company responses on 139 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P160.htm
Array lengths:  10 10 14 13
Company responses on 140 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P161.htm
Array lengths:  10 10 13 12
Company responses on 141 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P162.htm
Array lengths:  10 10 14 14
Company responses on 142 pages
Dataframe shape:  (200, 4)
https://www.glassdoor.com/Reviews/Liberty-Mutual-Insurance-Reviews-E2874_P

https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P6.htm
Array lengths:  10 10 10 10
Dataframe shape:  (60, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P7.htm
Array lengths:  10 10 10 10
Dataframe shape:  (70, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P8.htm
Array lengths:  10 10 10 10
Dataframe shape:  (80, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P9.htm
Array lengths:  10 10 10 10
Dataframe shape:  (90, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P10.htm
Array lengths:  10 10 10 10
Dataframe shape:  (100, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P11.htm
Array lengths:  10 10 10 10
Dataframe shape:  (110, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P12.htm
Array lengths:  10 10 10 10
Dataframe shape:  (120, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P13.htm
Array lengths:  10 10 10 10
Dataframe shape:  (130, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P14.htm
Array lengths:  10 

https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P77.htm
Array lengths:  10 10 10 10
Dataframe shape:  (760, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P78.htm
Array lengths:  10 10 10 10
Dataframe shape:  (770, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P79.htm
Array lengths:  10 10 10 10
Dataframe shape:  (780, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P80.htm
Array lengths:  10 10 10 10
Dataframe shape:  (790, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P81.htm
Array lengths:  10 10 10 10
Dataframe shape:  (800, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P82.htm
Array lengths:  10 10 10 10
Dataframe shape:  (810, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P83.htm
Array lengths:  10 10 10 10
Dataframe shape:  (820, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P84.htm
Array lengths:  10 10 10 10
Dataframe shape:  (830, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P85.htm
Array lengt

https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P147.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1460, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P148.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1470, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P149.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1480, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P150.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1490, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P151.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1500, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P152.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1510, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P153.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1520, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P154.htm
Array lengths:  10 10 10 10
Dataframe shape:  (1530, 4)
https://www.glassdoor.com/Reviews/Merck-Reviews-E438_P15

In [258]:
df.shape

(8558, 5)

### Text cleaning

In [230]:
df.pros = df.pros.map(lambda x: re.sub('\s+', ' ', x))

In [231]:
df.cons = df.cons.map(lambda x: re.sub('\s+', ' ', x))

In [232]:
# Export final dataframe
df.to_csv('../data/reviews_google.csv')

In [74]:
# df.title = df.title.map(lambda x: re.sub('[^a-zA-Z0-9 \n\.]', '', x.lower()))
# df.pros = df.pros.map(lambda x: re.sub('[^a-zA-Z0-9 \n\.]', '', x.lower()))
# df.cons = df.cons.map(lambda x: re.sub('[^a-zA-Z0-9 \n\.]', '', x.lower()))
df.head()

,title,rating,pros,cons,company
0,phenenomal career opportunity,5.0,environment work people benefits industry,long move from hometown nc,Exxon Mobil
1,great company that supports employees,5.0,great starting pay and benefits.,very bureaucratic and slow to adapt to change.,Exxon Mobil
2,great place to work,5.0,benefits are great insurance pension and 401k,the ranking system needs to be modified.,Exxon Mobil
3,administration assistant,5.0,great benefits work life balance positive company,i cant really name any cons. overall great com...,Exxon Mobil
4,part time job as a student,4.0,flexible hours for students looking for part t...,quite a lot of heavy lifting involved,Exxon Mobil


In [79]:
df.tail()

,title,rating,pros,cons,company
995,great place to work,2.0,stable company great opportunities hard workin...,work life balance not a key priority for this ...,Exxon Mobil
996,senior credit analyst prague,4.0,good starting position for fresh graduates en...,salary company believes in long term relation...,Exxon Mobil
997,data analyst,2.0,work life balance modern infrastructure,legacy model in management as well as in techn...,Exxon Mobil
998,best for rooting in longterm,4.0,incentive benefits welfare and environment is ...,routine job sometimes boring for new graduate,Exxon Mobil
999,work life balance company with moderate pay,3.0,work life balance is there. a lot of training ...,those who do the work are supervisor level bel...,Exxon Mobil
